## Getting Human Language Data

Here, we use an article on Nikola Tesla from history.com

In [ ]:
import requests
from bs4 import BeautifulSoup

def request():
    url = "https://www.history.com/topics/inventions/nikola-tesla"
    r = requests.get(url)
    text = r.text

    # create a beautiful soup object
    soup = BeautifulSoup(text, "html.parser")

    return soup


In [11]:
def scrape_info(soup):
    """
    :param soup: Beautiful soup object
    :return:
    """
    article_title = soup.find("h1", class_="m-detail-header--title")
    article_title = article_title.text

    article_body = soup.find("div", class_="m-detail--body")
    article_body.find("aside").decompose()
    article_body = article_body.text

    return article_title, article_body

print(scrape_info(request()))

('Nikola Tesla', 'Serbian-American engineer and physicist Nikola Tesla (1856-1943) made dozens of breakthroughs in the production, transmission and application of electric power. He invented the first alternating current (AC) motor and developed AC generation and transmission technology. Though he was famous and respected, he was never able to translate his copious inventions into long-term financial success—unlike his early employer and chief rival, Thomas Edison.Nikola Tesla’s Early Years     Nikola Tesla was born in 1856 in Smiljan, Croatia, then part of the Austro-Hungarian Empire. His father was a priest in the Serbian Orthodox church and his mother managed the family’s farm. In 1863 Tesla’s brother Daniel was killed in a riding accident. The shock of the loss unsettled the 7-year-old Tesla, who reported seeing visions—the first signs of his lifelong mental illnesses.Did you know? During the 1890s Mark Twain struck up a friendship with inventor Nikola Tesla. Twain often visited hi

In [34]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

## Tokenizing

In [28]:
from nltk.tokenize import sent_tokenize, word_tokenize

# Get data
title, article = scrape_info(request())

# sent_tokenize() to split up article into sentences:
sentences = sent_tokenize(article)

# tokenizing by word
article_words = word_tokenize(article)

In [20]:
sentences

['Serbian-American engineer and physicist Nikola Tesla (1856-1943) made dozens of breakthroughs in the production, transmission and application of electric power.',
 'He invented the first alternating current (AC) motor and developed AC generation and transmission technology.',
 'Though he was famous and respected, he was never able to translate his copious inventions into long-term financial success—unlike his early employer and chief rival, Thomas Edison.Nikola Tesla’s Early Years     Nikola Tesla was born in 1856 in Smiljan, Croatia, then part of the Austro-Hungarian Empire.',
 'His father was a priest in the Serbian Orthodox church and his mother managed the family’s farm.',
 'In 1863 Tesla’s brother Daniel was killed in a riding accident.',
 'The shock of the loss unsettled the 7-year-old Tesla, who reported seeing visions—the first signs of his lifelong mental illnesses.Did you know?',
 'During the 1890s Mark Twain struck up a friendship with inventor Nikola Tesla.',
 'Twain ofte

## Filtering Stop Words
Stop words are words that you want to ignore, so you filter them out of your text when you’re processing it. Very common words like 'in', 'is', and 'an' are often used as stop words since they don’t add a lot of meaning to a text in and of themselves.

In [30]:
from nltk.corpus import stopwords

# a list of english stop words
stop_words = stopwords.words("english")

# filter word tokens
filtered_words = [word for word in article_words if word not in stop_words]
print(filtered_words)

['Serbian-American', 'engineer', 'physicist', 'Nikola', 'Tesla', '(', '1856-1943', ')', 'made', 'dozens', 'breakthroughs', 'production', ',', 'transmission', 'application', 'electric', 'power', '.', 'He', 'invented', 'first', 'alternating', 'current', '(', 'AC', ')', 'motor', 'developed', 'AC', 'generation', 'transmission', 'technology', '.', 'Though', 'famous', 'respected', ',', 'never', 'able', 'translate', 'copious', 'inventions', 'long-term', 'financial', 'success—unlike', 'early', 'employer', 'chief', 'rival', ',', 'Thomas', 'Edison.Nikola', 'Tesla', '’', 'Early', 'Years', 'Nikola', 'Tesla', 'born', '1856', 'Smiljan', ',', 'Croatia', ',', 'part', 'Austro-Hungarian', 'Empire', '.', 'His', 'father', 'priest', 'Serbian', 'Orthodox', 'church', 'mother', 'managed', 'family', '’', 'farm', '.', 'In', '1863', 'Tesla', '’', 'brother', 'Daniel', 'killed', 'riding', 'accident', '.', 'The', 'shock', 'loss', 'unsettled', '7-year-old', 'Tesla', ',', 'reported', 'seeing', 'visions—the', 'first',

## Stemming

Stemming is a text processing task in which you reduce words to their root, which is the core part of a word. For example, the words “helping” and “helper” share the root “help.” Stemming allows you to zero in on the basic meaning of a word rather than all the details of how it’s being used. NLTK has more than one stemmer, but you’ll be using the Porter stemmer.

In [32]:
from nltk.stem import PorterStemmer

# stemming object
stemmer = PorterStemmer()

# stemmed_words
stemmed_words = [stemmer.stem(word) for word in filtered_words]

stemmed_words

['serbian-american',
 'engin',
 'physicist',
 'nikola',
 'tesla',
 '(',
 '1856-1943',
 ')',
 'made',
 'dozen',
 'breakthrough',
 'product',
 ',',
 'transmiss',
 'applic',
 'electr',
 'power',
 '.',
 'he',
 'invent',
 'first',
 'altern',
 'current',
 '(',
 'ac',
 ')',
 'motor',
 'develop',
 'ac',
 'gener',
 'transmiss',
 'technolog',
 '.',
 'though',
 'famou',
 'respect',
 ',',
 'never',
 'abl',
 'translat',
 'copiou',
 'invent',
 'long-term',
 'financi',
 'success—unlik',
 'earli',
 'employ',
 'chief',
 'rival',
 ',',
 'thoma',
 'edison.nikola',
 'tesla',
 '’',
 'earli',
 'year',
 'nikola',
 'tesla',
 'born',
 '1856',
 'smiljan',
 ',',
 'croatia',
 ',',
 'part',
 'austro-hungarian',
 'empir',
 '.',
 'hi',
 'father',
 'priest',
 'serbian',
 'orthodox',
 'church',
 'mother',
 'manag',
 'famili',
 '’',
 'farm',
 '.',
 'in',
 '1863',
 'tesla',
 '’',
 'brother',
 'daniel',
 'kill',
 'ride',
 'accid',
 '.',
 'the',
 'shock',
 'loss',
 'unsettl',
 '7-year-old',
 'tesla',
 ',',
 'report',
 'se

## Tagging Parts of Speech

Part of speech is a grammatical term that deals with the roles words play when you use them together in sentences. Tagging parts of speech, or POS tagging, is the task of labeling the words in your text according to their part of speech.

Here’s how to import the relevant parts of NLTK in order to tag parts of speech:

In [35]:
import nltk

nltk.pos_tag(filtered_words)

[('Serbian-American', 'JJ'),
 ('engineer', 'NN'),
 ('physicist', 'NN'),
 ('Nikola', 'NNP'),
 ('Tesla', 'NNP'),
 ('(', '('),
 ('1856-1943', 'JJ'),
 (')', ')'),
 ('made', 'VBD'),
 ('dozens', 'NNS'),
 ('breakthroughs', 'JJ'),
 ('production', 'NN'),
 (',', ','),
 ('transmission', 'NN'),
 ('application', 'NN'),
 ('electric', 'JJ'),
 ('power', 'NN'),
 ('.', '.'),
 ('He', 'PRP'),
 ('invented', 'VBD'),
 ('first', 'RB'),
 ('alternating', 'VBG'),
 ('current', 'JJ'),
 ('(', '('),
 ('AC', 'NNP'),
 (')', ')'),
 ('motor', 'NN'),
 ('developed', 'VBD'),
 ('AC', 'NNP'),
 ('generation', 'NN'),
 ('transmission', 'NN'),
 ('technology', 'NN'),
 ('.', '.'),
 ('Though', 'IN'),
 ('famous', 'JJ'),
 ('respected', 'VBN'),
 (',', ','),
 ('never', 'RB'),
 ('able', 'JJ'),
 ('translate', 'NN'),
 ('copious', 'JJ'),
 ('inventions', 'NNS'),
 ('long-term', 'JJ'),
 ('financial', 'JJ'),
 ('success—unlike', 'NN'),
 ('early', 'RB'),
 ('employer', 'NN'),
 ('chief', 'NN'),
 ('rival', 'NN'),
 (',', ','),
 ('Thomas', 'NNP'),
 (

All the words in the quote are now in a separate tuple, with a tag that represents their part of speech. But what do the tags mean? Here’s how to get a list of tags and their meanings:

In [38]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

## Lemmatizing

Now that you’re up to speed on parts of speech, you can circle back to lemmatizing. Like stemming, lemmatizing reduces words to their core meaning, but it will give you a complete English word that makes sense on its own instead of just a fragment of a word like 'discoveri'.

**Note:** A lemma is a word that represents a whole group of words, and that group of words is called a lexeme.

For example, if you were to look up the word “blending” in a dictionary, then you’d need to look at the entry for “blend,” but you would find “blending” listed in that entry.

In this example, “blend” is the lemma, and “blending” is part of the lexeme. So when you lemmatize a word, you are reducing it to its lemma.

Here’s how to import the relevant parts of NLTK in order to start lemmatizing

In [41]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
lemmatizer.lemmatize("breeding")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...


'breeding'